In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2 
import zipfile
import PIL.Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image 
from pathlib import Path 
from matplotlib import pyplot as plt 

# **Loading the data**

First we give the google colab the authority to access our google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

here we start uploading data in our Notebook: 

In [ ]:
  from PIL import Image
#Uploading data
TRAIN = "/content/drive/MyDrive/data/chest_xray/train"
TEST =  "/content/drive/MyDrive/data/chest_xray/test"
VAL = "/content/drive/MyDrive/data/chest_xray/val"



print('train' , len(os.listdir(TRAIN))) 
print('test' , len(os.listdir(TEST))) 
print('val' , len(os.listdir(VAL))) 


# path of specific image 
path='/content/drive/MyDrive/data/chest_xray/train/NORMAL/IM-0115-0001.jpeg'

#print image
plt.imshow(mpimg.imread(path))
plt.show()
#another way to print images 
plt.imshow(Image.open(path)) 




image = Image.open(path)
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)



# **Exploring the data**

In [ ]:
path = Path('')

In [ ]:
normal_image = path / '/content/drive/MyDrive/data/chest_xray/train/NORMAL/IM-0115-0001.jpeg'
pneumonia_image = path / '/content/drive/MyDrive/data/chest_xray/train/PNEUMONIA/person1000_bacteria_2931.jpeg'

In [ ]:
img = Image.open(normal_image)
imgp = Image.open(pneumonia_image)

In [ ]:
plt.imshow(img, cmap='bone')

In [ ]:
plt.imshow(imgp, cmap='bone')

In [ ]:
plt.imshow(img, cmap='jet')

In [ ]:
plt.imshow(imgp, cmap='jet')

In [ ]:
plt.imshow(img, cmap='inferno')

In [ ]:
plt.imshow(imgp, cmap='inferno')

In [ ]:
plt.imshow(img, cmap='gist_ncar')

In [ ]:
plt.imshow(imgp, cmap='gist_ncar')

# **pre-processing data**
Data preprocessing is the transformations applied to your data before feeding it to the model by converting the raw data into a clean data set.




**what we do?**

1-Data augmentation: is the technique of increasing the size of data used for training a model. We apply it on our data to make a better acurracy .

data augmentation techniques that we use are: Rotation, shift, shear, zoom, flip, fill.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'

)

then we applied the following steps:

2-read the picture files through `flow_from_directory`function. 

3-rescale the pixel values (between 0 and 255) to [0,1] interval(normalization).

4-resize image to 350x350.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True,
    )
train_generator = train_datagen.flow_from_directory(
        TRAIN,# This is the target directory
        target_size=(350, 350), # as large as possible    
        batch_size=32,# No. of images to be yielded from the generator per batch.
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')# Set “binary” if you have only two classes to predict,
validation_generator =test_datagen.flow_from_directory(
        VAL,
        target_size=(350, 350),
        batch_size=32,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        TEST,
        target_size=(350, 350),
        batch_size=32,
        class_mode='binary')
        #shuffle=False)

**here we print the `data_batch` and `labels_batch`:**



In [ ]:

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break


# **Building the model**
we chose the type of keras models which is sequential model, sequential is linear stack of layers.

building the layers :

**step1:** Convolution `layers.Conv2D` A convolution multiplies a matrix of pixels with a filter matrix or ‘kernel’ and sums up the multiplication values.
Then the convolution slides over to the next pixel and repeats the same process until all the image pixels have been covered and generate feature maps. 

**step2:** pooling which is down-sampling, most often in the form of "max-pooling," `layers.MaxPooling2D`
where we select a region, and then take the maximum value in that region, and that becomes the new value for the entire region. 

**then we repeat these two steps three times with increasing the number of nodes on convolution layer to have more details.**

**step3:**Flatten `layers.Flatten()` this converts our 3D feature maps to 1D feature vectors.

**step4:**Dropout `layers.Dropout`prevent neural networks from Overfitting by reducing the capacity or thinning the network during training.

**step5:**Dense `layers.Dense`
is the layer that we will use in for our output layer.we use two Dense layers 



In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model


model = models.Sequential() 
#architecture of the model

#step1 #32 is number of nodes in layer#(3,3) is the size of the window 
model.add(layers.Conv2D(32, (3, 3), activation='relu', #32 number of nodes in the layer.
                        input_shape=(350, 350, 3))) #(rows, cols, RBG)
                        
#step2 #(2,2) is the size of window OR (Kernel size) 
model.add(layers.MaxPooling2D((2, 2))) 
                           

#another layer of Convolution
model.add(layers.Conv2D(64, (3, 3), activation='relu'))  
                                                        

#another layer of pooling
model.add(layers.MaxPooling2D((2, 2)))
#another layer of Convolution
model.add(layers.Conv2D(128, (3, 3), activation='relu')) 

#another layer of pooling
model.add(layers.MaxPooling2D((2, 2))) 

#step3
model.add(layers.Flatten())

#step4
model.add(layers.Dropout(0.5))

#step5
model.add(layers.Dense(512, activation='relu')) #521 number of nodes in the layer.
model.add(layers.Dense(1, activation='sigmoid')) #1 number of nodes in the layer.

#Before training a model, you need to configure the learning process,
# which is done via the compile() function.


history= model.compile(loss='binary_crossentropy',
              optimizer='adam',#The optimizer controls the learning rate  
             metrics=['binary_accuracy']
              )

We display layers into two ways:

1-Graphical 

2- Table 

From these two ways we can observe that each layer’s output becomes the input for the subsequent layer.

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Each layer has an output and its shape is shown in the `Output Shape` column. 

`output shape`: batch_shape + (new_rows, new_cols, filters).

`Param`: column shows you the number of parameters that are trained for each layer.

In [ ]:
model.summary() 

Now we will train the model using `model.fit`:

In [ ]:
history=model.fit(
    train_generator,
    steps_per_epoch=60,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/30
60/60 [==============================] - 798s 13s/step - loss: 1.3516 - binary_accuracy: 0.6789 - val_loss: 0.5729 - val_binary_accuracy: 0.5625
Epoch 2/30
60/60 [==============================] - 613s 10s/step - loss: 0.3920 - binary_accuracy: 0.8090
Epoch 3/30
60/60 [==============================] - 605s 10s/step - loss: 0.3594 - binary_accuracy: 0.8165
Epoch 4/30
60/60 [==============================] - 576s 10s/step - loss: 0.3613 - binary_accuracy: 0.8363
Epoch 5/30
60/60 [==============================] - 570s 9s/step - loss: 0.3091 - binary_accuracy: 0.8648
Epoch 6/30
45/60 [=====================>........] - ETA: 2:18 - loss: 0.2775 - binary_accuracy: 0.8814

## **Evaluate the model**

In [ ]:
#Try to install HDF5 format with this line:

# !pip install pyyaml h5py
from keras.models import load_model
from tensorflow.keras.models import save_model, load_model
save_model("pneumonia_model.h5")
model = load_model("pneumonia_model.h5")

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
test_loss, test_accuracy = model.evaluate(test_generator,verbose=0)
test_loss = round(test_loss, 6)
test_accuracy = round(test_accuracy*100, 3)

print('Test Loss: ', test_loss)
print('Test Accuracy:', '\033[0m', test_accuracy, '%\033[0m')

In [ ]:
# Confusion Matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
batch_size=32
Y_pred = model.predict(test_generator, 624 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
confusion_mtx=confusion_matrix(test_generator.classes, y_pred)
#plot_confusion_matrix(estimator=classifier, X = test_generator.classes, y_true = y_pred,labels= range(2), normalize=False)
print('Classification Report')
target_names = ['Normal', 'Pneumonia']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))